# Data Gathering

In [ ]:
#dati aggiornati al 2 maggio

import json
import os
import requests
from bs4 import BeautifulSoup
import networkx as nx
from statsmodels.distributions.empirical_distribution import ECDF
import numpy as np
import matplotlib.pyplot as plt

twitch_keys = json.load(open('json/keys.json','r'))['twitch']
client_id = twitch_keys['client_id']
client_secret = twitch_keys['client_secret']

#access token
url = 'https://id.twitch.tv/oauth2/token?client_id='+client_id+'&client_secret='+client_secret+'&grant_type=client_credentials'
reply = requests.post(url)
access_token = reply.json()['access_token']
print(access_token)
data = {'Client-ID': client_id, 'Authorization': 'Bearer ' + access_token}

In [ ]:
def getFollowersById(id_twitcher):
    cursor = ""
    ids = []
    
    for i in range(100):
        url = 'https://api.twitch.tv/helix/users/follows?to_id='+id_twitcher+'&first=100&after='+cursor
        reply = requests.get(url, headers=data)
        print("{} richiesta inviata".format(i))
        if len(reply.json()['pagination'])==0:
            break
        else:
            cursor = reply.json()['pagination']['cursor']
            for x in reply.json()['data']:
                ids.append(x['from_id'])        
    return(ids)

def most_common(lst):
    return max(set(lst), key=lst.count)

In [ ]:
#top 500 canali con più followers
url = 'https://socialblade.com/twitch/top/500/followers'
headers_update = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 ' + 
    ' (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}
html_reply = requests.get(url, headers=headers_update).text
html_soup = BeautifulSoup(html_reply, 'html.parser')


header = []
for found in html_soup.select('.table-header'):
    header.append(found.text.replace('•', ''))
print(header)


cells_html = html_soup.select('.table-cell')
channels = []
for i in range(0, 2500, 5):
    x = dict()
    for h in header:    
        x[h] = cells_html[i].text
        i += 1
    channels.append(x)

channels_name_flw = []
for x in channels:
    channels_name_flw.append({'name': x['Username'], 'followers': x['Followers']})
print(channels_name_flw)

with open('json/channels_name_flw.json', 'w') as json_file:
    json.dump(channels_name_flw, json_file)

In [ ]:
channels_id_json = []       
print(data)
for i in range(0, 500):
    url = 'https://api.twitch.tv/helix/users?login=' + channels_name[i]
    reply = requests.get(url, headers=data)
    channels_id_json.append(reply.json())
    print(i)
    print(channels_id_json[i])
    

In [ ]:
print(len(channels_id_json))

In [ ]:
with open('json/channels_id_json.json', 'w') as json_file:
    json.dump(channels_id_json, json_file)

In [ ]:
print("\nCHANNELS ID")

with open('json/channels_id_json.json') as json_file:
    channels_id_json = json.load(json_file)
print(channels_id_json)

channels_id = []
idAndName = []
for x in channels_id_json:
    if x['data']:
        channels_id.append(x['data'][0]['id'])
        idAndName.append({'id': x['data'][0]['id'], 'name': x['data'][0]['display_name']})
for _ in range(8):
    del channels_id[-1]
print(len(channels_id))

with open('json/idAndName.json', 'w') as json_file:
    json.dump(idAndName, json_file)

In [ ]:
for i in range(500):
    print(i)
    json_obj = {'id':idAndName[i]['id'], 'name':idAndName[i]['name'], 'followers':getFollowersById(idAndName[i]['id'])}
    with open('json/allFollowers.json', 'a') as f:
        json.dump(json_obj, f)
        f.write(os.linesep)

In [ ]:
flw = []
for i in range(500):
    print(i)
    flw.append({'id':idAndName[i]['id'], 'name':idAndName[i]['name'], 'followers':getFollowersById(idAndName[i]['id'])})

print("\n\n\n")
    
intersect = []    
for i in range(500):
    print(i)
    set1 = set(flw[i]['followers'])
    for j in range(500):
         if i!=j:
            set2 = set(flw[j]['followers'])
            set3 = set1.intersection(set2)
            intersect.append({'id1': flw[i]['id'], 'name1': flw[i]['name'],
                              'id2': flw[j]['id'], 'name2': flw[j]['name'],
                              'common_flw': len(set3)
                             })

with open('json/intersect.json', 'w') as json_file:
    json.dump(intersect, json_file)

In [ ]:
for x in intersect:
    with open('data/edge_list.txt', 'a') as f:
        if x['common_flw'] != 0:
            line = x['id1'] + " " + x['id2'] + " " + str(x['common_flw'])
            f.write(line)
            f.write(os.linesep)

# Top categories

In [ ]:
topGameStreamer = []

for i in range(0, len(channels_id)):
    games = []
    url = 'https://api.twitch.tv/helix/clips?broadcaster_id=' + channels_id[i] + '&first=100'
    reply = requests.get(url, headers=data)
    print(i)
    if reply.json()['data']:
        for y in reply.json()['data']:
            url2 = 'https://api.twitch.tv/helix/games?id=' + y['game_id']
            reply2 = requests.get(url2, headers=data)
            if reply2.json()['data']:
                games.append(reply2.json()['data'][0]['name'])
        topGameStreamer.append({'streamer': channels_id[i], 'top_game': most_common(games)})
    else: 
        topGameStreamer.append({'streamer': channels_id[i], 'top_game': "NA"})

In [ ]:
with open('json/top_game.json', 'w') as json_file:
    json.dump(topGameStreamer, json_file)        